In [1]:
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import torch
from IPython.display import display
import torch.nn as nn
from torchvision.utils import make_grid
from torchvision.utils import save_image
from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
from tqdm import tqdm_notebook
import seaborn as sns
import matplotlib.pyplot as plt
from torch.nn.modules.activation import ReLU
from torch.optim import Adam
from tqdm import tqdm_notebook
from torchvision.utils import save_image
import matplotlib
import math
from inspect import isfunction
from functools import partial
import scipy
from scipy.special import rel_entr
from torch import nn, einsum
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from einops import rearrange
from einops.layers.torch import Rearrange
from torch import nn, einsum
import torch.nn.functional as F
import matplotlib.animation as animation
import matplotlib.image as mpimg
import glob
from PIL import Image
from itertools import product
import random
from FID_helper import *
import random
from scipy.linalg import sqrtm
%matplotlib inline

In [26]:
def load_data(dataset_name, seq_len, sample_num):
    seq = np.load(dataset_name, allow_pickle=True)
    seq = seq.tolist()
    random.seed(3)
    seq_sample = random.sample(seq, sample_num)

    return  seq2onehot(seq_sample, seq_len)


def get_1d_fid_score(syn_seq_name, nat_seq_name, model_1d, seq_len):
    sample_num = 1000
    syn_seq = load_data(syn_seq_name, seq_len, sample_num)
    nat_seq = load_data(nat_seq_name, seq_len, sample_num)

    nat_seq_1d = nat_seq.reshape(sample_num, 4, seq_len)
    nat_seq_1d = train_dataset(nat_seq_1d)
    nat_seq_1d = DataLoader(nat_seq_1d)

    syn_seq_1d = syn_seq.reshape(sample_num, 4, seq_len)
    syn_seq_1d = train_dataset(syn_seq_1d)
    syn_seq_1d = DataLoader(syn_seq_1d)

    fea_nat_1d = np.zeros((1,256), dtype=float)
    for data in nat_seq_1d:
        fea = model_1d(data['feature'] )
        fea_nat_1d = np.append(fea_nat_1d, fea.detach().numpy(), axis=0)
    fea_nat_1d = np.delete(fea_nat_1d, 0, axis=0)

    fea_syn_1d = np.zeros((1,256), dtype=float)
    for data in syn_seq_1d:
        fea = model_1d(data['feature'] )
        fea_syn_1d = np.append(fea_syn_1d, fea.detach().numpy(), axis=0)
    fea_syn_1d = np.delete(fea_syn_1d, 0, axis=0)

    mus_nat_1d = fea_nat_1d.mean(axis=0)
    mus_syn_1d = fea_syn_1d.mean(axis=0)
    var_nat_1d = np.cov(fea_nat_1d , rowvar=False)
    var_syn_1d = np.cov(fea_syn_1d , rowvar=False)

    ssdiff = np.sum((mus_nat_1d - mus_syn_1d)**2.0)
    print(f"ssdiff {ssdiff}")
    covmean = sqrtm(var_nat_1d.dot(var_syn_1d))
    print(f"covmean {covmean}")
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid_1d = ssdiff + np.trace(var_nat_1d + var_syn_1d - 2.0 * covmean)

    return  fid_1d


def get_2d_fid_score(syn_seq_name, nat_seq_name, seq_len, model_2d, dim):
    sample_num = 1000
    syn_seq = load_data(syn_seq_name, seq_len, sample_num)
    nat_seq = load_data(nat_seq_name, seq_len, sample_num)

    nat_seq_2d = nat_seq.reshape(sample_num, seq_len, 4, 1)
    nat_seq_2d = train_dataset(nat_seq_2d)
    nat_seq_2d = DataLoader(nat_seq_2d, batch_size=1024)

    syn_seq_2d = syn_seq.reshape(sample_num, seq_len, 4, 1)
    syn_seq_2d = train_dataset(syn_seq_2d)
    syn_seq_2d = DataLoader(syn_seq_2d, batch_size=1024)

    # model_2d = PREDICT_2D()
    # model_2d.load_state_dict(torch.load(model_name))

    fea_nat_2d = np.zeros((1, dim), dtype=float)
    for data in nat_seq_2d:
        fea = model_2d(data['feature'])
        fea_nat_2d = np.append(fea_nat_2d, fea.detach().numpy(), axis=0)
    fea_nat_2d = np.delete(fea_nat_2d, 0, axis=0)

    fea_syn_2d = np.zeros((1, dim), dtype=float)
    for data in syn_seq_2d:
        fea = model_2d(data['feature'] )
        fea_syn_2d = np.append(fea_syn_2d, fea.detach().numpy(), axis=0)
    fea_syn_2d = np.delete(fea_syn_2d, 0, axis=0)

    mus_nat_2d = fea_nat_2d.mean(axis=0)
    mus_syn_2d = fea_syn_2d.mean(axis=0)
    var_nat_2d = np.cov(fea_nat_2d , rowvar=False)
    var_syn_2d = np.cov(fea_syn_2d , rowvar=False)

    ssdiff = np.sum((mus_nat_2d - mus_syn_2d)**2.0)
    # print(f"ssdiff {ssdiff}")
    covmean = sqrtm(var_nat_2d.dot(var_syn_2d))
    # print(f"covmean {covmean}")
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid_2d = ssdiff + np.trace(var_nat_2d + var_syn_2d - 2.0 * covmean)
    # print(fid_2d)
    return fid_2d


In [3]:
class PREDICT_1(nn.Module):

    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(50, 100, kernel_size=(6, 1), padding=(3, 0)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(100, 200, kernel_size=(5, 1), padding=(3, 0)),
            nn.ReLU()
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(200, 200, kernel_size=(6, 1), padding=(3, 0)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 1)),
            nn.Flatten()
        )
        self.fc = nn.Sequential(
            nn.Linear(400, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        # x = self.fc(x)
        # x = x.squeeze(-1)
        # x = x.squeeze(-1)
        return x

model = PREDICT_1()
model.load_state_dict(torch.load('fid_model/promoter_1/CNN_train.pth'))
dif_fid = []
gan_fid = []
for i in range(130):
    if i % 25 == 24:
        dif_fid.append(get_2d_fid_score("syn_seq/promoter_1/seq_epoch_{}.npy".format(i), "data/promoter_1.npy", 50, model, 400))
        gan_fid.append(get_2d_fid_score("gan_seq/promoter_1/seq_epoch_{}.npy".format(i), "data/promoter_1.npy", 50, model, 400))





FileNotFoundError: [Errno 2] No such file or directory: 'syn_seq/promoter_1/seq_epoch_24.npy'

In [5]:
class PREDICT_2(nn.Module):

    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(165, 300, kernel_size=(6, 1), padding=(3, 0)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(300, 600, kernel_size=(5, 1), padding=(3, 0)),
            nn.ReLU()
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(600, 600, kernel_size=(6, 1), padding=(3, 0)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 1)),
            nn.Flatten()
        )
        self.fc = nn.Sequential(
            nn.Linear(1200, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        # x = self.fc(x)
        # x = x.squeeze(-1)
        # x = x.squeeze(-1)
        return x

model = PREDICT_2()
model.load_state_dict(torch.load('fid_model/promoter_2/CNN_train.pth'))
dif_fid = []
gan_fid = []
for i in range(130):
    if i % 25 == 24:
        dif_fid.append(get_2d_fid_score("syn_seq/promoter_2/seq_epoch_{}.npy".format(i), "data/promoter_2.npy", 165, model, 1200))
        gan_fid.append(get_2d_fid_score("gan_seq/promoter_2/seq_epoch_{}.npy".format(i), "data/promoter_2.npy", 165, model, 1200))


In [7]:
dif_fid


[0.14667508270770965,
 0.1371422848951489,
 0.15874560439821428,
 0.13899011367347583,
 0.13647543427678344]

In [8]:
gan_fid

[0.34555065855256617,
 0.21908250867339132,
 0.21942269444060641,
 0.21668955356713365,
 0.22539711609990742]

In [9]:
class PREDICT_3(nn.Module):

    # 卷积层结构卷积-ReLU-池化-卷积-ReLU-卷积-ReLU-池化-全连接-ReLU-全连接
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(118, 200, kernel_size=(6, 1), padding=(3, 0)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(200, 400, kernel_size=(5, 1), padding=(3, 0)),
            nn.ReLU()
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(400, 400, kernel_size=(6, 1), padding=(3, 0)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 1)),
            nn.Flatten()
        )
        self.fc = nn.Sequential(
            nn.Linear(800, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        # x = self.fc(x)
        # x = x.squeeze(-1)
        # x = x.squeeze(-1)
        return x

model = PREDICT_3()
model.load_state_dict(torch.load('fid_model/promoter_3/CNN_train.pth'))
dif_fid = []
gan_fid = []
for i in range(130):
    if i % 25 == 24:
        dif_fid.append(get_2d_fid_score("syn_seq/promoter_3/seq_epoch_{}.npy".format(i), "data/promoter_3.npy", 118, model, 800))
        gan_fid.append(get_2d_fid_score("gan_seq/promoter_3/seq_epoch_{}.npy".format(i), "data/promoter_3.npy", 118, model, 800))


In [10]:
dif_fid

[0.14602551439108105,
 0.6514319866414572,
 0.14433246545097428,
 0.1128892669418097,
 0.15756700881187558]

In [11]:
gan_fid

[0.9759876493436314,
 0.9803591615712742,
 1.2300523329491055,
 1.2796727514867205,
 1.6413831349875943]

In [12]:
class PREDICT(nn.Module):

    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(50, 100, kernel_size=(6, 1), padding=(3, 0)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(100, 200, kernel_size=(5, 1), padding=(3, 0)),
            nn.ReLU()
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(200, 200, kernel_size=(6, 1), padding=(3, 0)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 1)),
            nn.Flatten()
        )
        self.fc = nn.Sequential(
            nn.Linear(400, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        # x = self.fc(x)
        # x = x.squeeze(-1)
        # x = x.squeeze(-1)
        return x


model = PREDICT_1()
model.load_state_dict(torch.load('fid_model/promoter_1/CNN_train.pth'))
dif_fid = []
gan_fid = []
for i in range(130):
    if i % 25 == 24:
        dif_fid.append(get_2d_fid_score("syn_seq/promoter_1/seq_epoch_{}.npy".format(i), "data/promoter_1.npy", 50, model, 400))
        gan_fid.append(get_2d_fid_score("gan_seq/promoter_1/seq_epoch_{}.npy".format(i), "data/promoter_1.npy", 50, model, 400))

In [13]:
dif_fid

[0.018770344171487766,
 0.020423965282888923,
 0.018673418506755985,
 0.019368654406041647,
 0.018937890635125194]

In [14]:
gan_fid

[0.02863051575390797,
 0.019944700622179422,
 0.02190636098476251,
 0.020911923764308893,
 0.021484443022301743]

In [ ]:
dif_fid_list = [[0.018770344171487766,
                 0.020423965282888923,
                 0.018673418506755985,
                 0.019368654406041647,
                 0.018937890635125194],
                ]

In [27]:
class PREDICT_1(nn.Module):

    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(50, 100, kernel_size=(6, 1), padding=(3, 0)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(100, 200, kernel_size=(5, 1), padding=(3, 0)),
            nn.ReLU()
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(200, 200, kernel_size=(6, 1), padding=(3, 0)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 1)),
            nn.Flatten()
        )
        self.fc = nn.Sequential(
            nn.Linear(400, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        # x = self.fc(x)
        # x = x.squeeze(-1)
        # x = x.squeeze(-1)
        return x


class PREDICT_2(nn.Module):

    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(165, 300, kernel_size=(6, 1), padding=(3, 0)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(300, 600, kernel_size=(5, 1), padding=(3, 0)),
            nn.ReLU()
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(600, 600, kernel_size=(6, 1), padding=(3, 0)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 1)),
            nn.Flatten()
        )
        self.fc = nn.Sequential(
            nn.Linear(1200, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        # x = self.fc(x)
        # x = x.squeeze(-1)
        # x = x.squeeze(-1)
        return x


class PREDICT_3(nn.Module):

    # 卷积层结构卷积-ReLU-池化-卷积-ReLU-卷积-ReLU-池化-全连接-ReLU-全连接
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(118, 200, kernel_size=(6, 1), padding=(3, 0)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 1))
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(200, 400, kernel_size=(5, 1), padding=(3, 0)),
            nn.ReLU()
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(400, 400, kernel_size=(6, 1), padding=(3, 0)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 1)),
            nn.Flatten()
        )
        self.fc = nn.Sequential(
            nn.Linear(800, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        # x = self.fc(x)
        # x = x.squeeze(-1)
        # x = x.squeeze(-1)
        return x


model_1 = PREDICT_1()
model_1.load_state_dict(torch.load('fid_model/promoter_1/CNN_train.pth'))

model_2 = PREDICT_2()
model_2.load_state_dict(torch.load('fid_model/promoter_2/CNN_train.pth'))

model_3 = PREDICT_3()
model_3.load_state_dict(torch.load('fid_model/promoter_3/CNN_train.pth'))

seq_len = [50, 165, 118]

dim = [400, 1200, 800]

dif_fid = []
gan_fid = []

e = 1
gan_fid_temp = []
dif_fid_temp = []
print("promoter_{}".format(e))
print("ep.\t gan\t dif")
for i in range(130):
        if i % 10 == 9:
            dif_fid_temp.append(get_2d_fid_score("syn_seq/promoter_{}/new_seq_epoch_{}.npy".format(e, i), "data/promoter_{}.npy".format(e), seq_len[e-1], model_1, dim[e-1]))
            gan_fid_temp.append(get_2d_fid_score("gan_seq/promoter_{}/samples_{}.npy".format(e, i), "data/promoter_{}.npy".format(e), seq_len[e-1], model_1, dim[e-1]))
            print("{}\t {:.4f}\t {:.4f}".format(i+1, gan_fid_temp[-1], dif_fid_temp[-1]))
gan_fid.append(gan_fid_temp)
dif_fid.append(dif_fid_temp)


e = 2
gan_fid_temp = []
dif_fid_temp = []
print("promoter_{}".format(e))
print("ep.\t gan\t dif")
for i in range(130):
        if i % 10 == 9:
            dif_fid_temp.append(get_2d_fid_score("syn_seq/promoter_{}/new_seq_epoch_{}.npy".format(e, i), "data/promoter_{}.npy".format(e), seq_len[e-1], model_2, dim[e-1]))
            gan_fid_temp.append(get_2d_fid_score("gan_seq/promoter_{}/samples_{}.npy".format(e, i), "data/promoter_{}.npy".format(e), seq_len[e-1], model_2, dim[e-1]))
            print("{}\t {:.4f}\t {:.4f}".format(i+1, gan_fid_temp[-1], dif_fid_temp[-1]))
gan_fid.append(gan_fid_temp)
dif_fid.append(dif_fid_temp)

e = 3
gan_fid_temp = []
dif_fid_temp = []
print("promoter_{}".format(e))
print("ep.\t gan\t dif")
for i in range(130):
        if i % 10 == 9:
            dif_fid_temp.append(get_2d_fid_score("syn_seq/promoter_{}/new_seq_epoch_{}.npy".format(e, i), "data/promoter_{}.npy".format(e), seq_len[e-1], model_3, dim[e-1]))
            gan_fid_temp.append(get_2d_fid_score("gan_seq/promoter_{}/samples_{}.npy".format(e, i), "data/promoter_{}.npy".format(e), seq_len[e-1], model_3, dim[e-1]))
            print("{}\t {:.4f}\t {:.4f}".format(i+1, gan_fid_temp[-1], dif_fid_temp[-1]))
gan_fid.append(gan_fid_temp)
dif_fid.append(dif_fid_temp)


promoter_1
ep.	 gan	 dif
10	 0.0840	 0.0197
20	 0.0275	 0.0190
30	 0.0234	 0.0191
40	 0.0250	 0.0185
50	 0.0202	 0.0187
60	 0.0205	 0.0183
70	 0.0198	 0.0182
80	 0.0207	 0.0185
90	 0.0200	 0.0187
100	 0.0199	 0.0187
110	 0.0202	 0.0183
120	 0.0200	 0.0194
130	 0.0216	 0.0186
promoter_2
ep.	 gan	 dif
10	 11.6918	 0.1816
20	 0.2844	 0.2169
30	 0.3218	 0.1390
40	 0.3766	 0.1641
50	 0.4111	 0.1394
60	 0.1870	 0.1893
70	 0.2056	 0.1467
80	 0.1894	 0.1484
90	 0.1772	 0.1771
100	 0.2241	 0.1398
110	 0.1907	 0.1408
120	 0.1747	 0.1720
130	 0.2097	 0.1372
promoter_3
ep.	 gan	 dif
10	 1.7199	 0.5522
20	 0.7258	 0.2623
30	 0.4541	 0.2406
40	 1.1713	 0.1882
50	 0.9300	 0.6210
60	 0.5698	 0.3733
70	 0.7089	 0.1547
80	 0.8405	 0.1237
90	 1.0051	 0.1357
100	 0.9378	 0.0958
110	 0.7684	 0.3367
120	 0.7943	 0.4636
130	 0.7276	 0.1597
